# Pockets of data from different data sets are integrated, and only the most important four atoms of each amino acid are retained

In [1]:
import os
import numpy as np
import pandas as pd
import re
import sys

In [2]:
#true
#按氨基酸序号来分，得到对应的类型
def save_pocket(save_name_pocket,anno_floder_path,pdb_file):
    with open(save_name_pocket,'w') as ff:
        label = "str"
        number = 0
        with open(os.path.join(anno_floder_path,pdb_file)) as f:
            lines= f.readline()
            line = lines.strip()
            while line:
                if line[23:26]!=label:
                    ff.write(lines)
                    label = line[23:26]
                    number =0
                elif line[23:26]==label and number<3:
                    ff.write(lines)
                    number+=1
                lines= f.readline()
                line = lines.strip()

In [3]:
#true ，get dir and file
#没有写新complex
path = ['./pocket6A/','./pocket_pepbdb6A/']
for p in path:
    print(p)
    now_num=0
    all_num=len(os.listdir(p))
    for pdb_file in os.listdir(p):
        now_num=now_num+1
        print("\r", end="")
        print("Download progress: {}% ".format(now_num/all_num*100), end="")
        sys.stdout.flush()
        save_name_pocket='./pocket6A_gvp/'+ os.path.splitext(pdb_file)[0] + '.txt'
        if os.path.exists(save_name_pocket):
            #print('already exist')
            x=[]
        else:
            save_pocket(save_name_pocket,p,pdb_file)

./pocket6A/
Download progress: 100.0% 821526593%     ./pocket_pepbdb6A/
Download progress: 100.0% 0173833486%    

# Input of esm-if1 in order to obtain the features of esm-if1

In [49]:
import jsonlines
import numpy as np
import pandas as pd
import os

In [50]:
def name_to_simple(name):
    names = {
        "GLY" : 'G',
        "ALA" : 'A',
        "VAL" : 'V',
        "LEU" : 'L',
        "ILE" : 'I',
        "PRO" : 'P',
        "PHE" : 'F',
        "TYR" : 'Y',
        "TRP" : 'W',
        "SER" : 'S',
        "THR" : 'T',
        "CYS" : 'C',
        "MET" : 'M',
        "ASN" : 'N',
        "GLN" : 'Q',
        "ASP" : 'D',
        "GLU" : 'E',
        "LYS" : 'K',
        "ARG" : 'R',
        "HIS" : 'H'}
    return names.get(name,None)

In [44]:
#判断表示=链名+氨基酸序列号
with jsonlines.open('./gvp_json/esmif_data6A.jsonl', mode='w') as writer:  
    path = ['./pocket6A_gvp']
    save_name =[]
    for p in path:
        now_num=0
        all_num=len(os.listdir(p))
        for pdb_file in os.listdir(p):
            now_num=now_num+1
#             print("\r", end="")
#             print("Download progress: {}% ".format(now_num/all_num*100), end="")
#             sys.stdout.flush()
            count_number = 0
            resID=[]
            resList=[]
            resCoors = []
            pdb_data = {}
            resCoor =[]
            flag = 1
    #         xxx=1
    #         yyy = 1
            with open(os.path.join(p,pdb_file)) as f:
                lines= f.readline()
                line = lines.strip()
                #line_split = line.split()

                while line:
                    x = name_to_simple(line[17:20])
                    if x !=None and line[0:4] =="ATOM":
                        if line[21:26] not in resID:
                            resList.append(x)
    #                         if xxx!=yyy:
    #                             print(yyy)
    #                         xxx +=4
                            resID.append(line[21:26])

                        rescoor = []
                        x1,y,z =float(line[30:38]),float(line[38:46]),float(line[46:54])
                        rescoor.append(x1)
                        rescoor.append(y)
                        rescoor.append(z)
                       

                        count_number+=1
                        if count_number<4:
                            resCoor.append(rescoor)
                        elif count_number==4:
                            resCoors.append(resCoor)
                            count_number =0 
                            resCoor = []
    #                 if len(resList)!=len(resCoors):
    #                     print(len(resList))
    #                     print(len(resCoors))
                    lines= f.readline()
                    line = lines.strip()
    #                yyy+=1
                    #line_split = line.split()
            pdb_data['name'] = pdb_file
            pdb_data_seq = "".join(resList)
            pdb_data['coords'] = resCoors
            if len(pdb_data_seq)==len(resCoors) and len(pdb_data_seq)!=0:
                writer.write(pdb_data)
            else:
                with open("./eooro.txt","a+") as errorf:
                    errorf.write("\n")
                    errorf.write(pdb_file)
#                 print("error in length")
#                 print(pdb_data['name'])
#                 print(len(pdb_data_seq))
#                 print(len(resCoors))

In [45]:
#前6000个，中间6000个，最后分成part1，2，3
num = 0
with jsonlines.open('./gvp_json/esmif_data6A_part1.jsonl', mode='w') as writer: 
    with open("./gvp_json/esmif_data6A.jsonl", "r+", encoding="utf8") as f:
        for item in jsonlines.Reader(f):
            num+=1
            if num>=0 and num<=6000:
                writer.write(item)
num = 0
with jsonlines.open('./gvp_json/esmif_data6A_part2.jsonl', mode='w') as writer: 
    with open("./gvp_json/esmif_data6A.jsonl", "r+", encoding="utf8") as f:
        for item in jsonlines.Reader(f):
            num+=1
            if num>=6001 and num<=12000:
                writer.write(item)
num = 0
with jsonlines.open('./gvp_json/esmif_data6A_part3.jsonl', mode='w') as writer: 
    with open("./gvp_json/esmif_data6A.jsonl", "r+", encoding="utf8") as f:
        for item in jsonlines.Reader(f):
            num+=1
            if num>=12001:
                writer.write(item)
        

In [46]:
num = 0
with open("./gvp_json/esmif_data6A.jsonl", "r+", encoding="utf8") as f:
    for item in jsonlines.Reader(f):
        num+=1
        print(item["name"])
        print(np.array(item["coords"]).shape)

148l_S_E.txt
(9, 3, 3)
1a07_C_A.txt
(6, 3, 3)
1a07_D_B.txt
(6, 3, 3)
1a08_C_A.txt
(7, 3, 3)
1a08_D_B.txt
(5, 3, 3)
1a09_C_A.txt
(6, 3, 3)
1a09_D_B.txt
(7, 3, 3)
1a0m_A.txt
(6, 3, 3)
1a0n_A.txt
(11, 3, 3)
1a1a_C_A.txt
(7, 3, 3)
1a1a_D_B.txt
(7, 3, 3)
1a1b_C_A.txt
(6, 3, 3)
1a1b_D_B.txt
(6, 3, 3)
1a1c_C_A.txt
(6, 3, 3)
1a1c_D_B.txt
(6, 3, 3)
1a1e_C_A.txt
(6, 3, 3)
1a1e_D_B.txt
(5, 3, 3)
1a1m_C_A.txt
(41, 3, 3)
1a1n_C_A.txt
(41, 3, 3)
1a1o_C_A.txt
(47, 3, 3)
1a1r_C_A.txt
(46, 3, 3)
1a1r_D_A.txt
(72, 3, 3)
1a1u_A.txt
(19, 3, 3)
1a2c_I_H.txt
(16, 3, 3)
1a2c_J_H.txt
(7, 3, 3)
1a2c_L_H.txt
(50, 3, 3)
1a2x_B_A.txt
(40, 3, 3)
1a30_C_A.txt
(25, 3, 3)
1a37_P.txt
(10, 3, 3)
1a38_P.txt
(20, 3, 3)
1a3b_I_H.txt
(15, 3, 3)
1a3b_L_H.txt
(39, 3, 3)
1a3e_I_H.txt
(17, 3, 3)
1a3e_L_H.txt
(40, 3, 3)
1a3i_A.txt
(12, 3, 3)
1a3i_B.txt
(14, 3, 3)
1a3j_A.txt
(12, 3, 3)
1a3j_B.txt
(14, 3, 3)
1a3r_P_H.txt
(39, 3, 3)
1a46_I_H.txt
(15, 3, 3)
1a46_L_H.txt
(37, 3, 3)
1a4w_I_H.txt
(15, 3, 3)
1a4w_L_H.txt
(38, 3, 3)
1a5

1k7l_F_C.txt
(33, 3, 3)
1k7l_H_E.txt
(22, 3, 3)
1k83_M.txt
(12, 3, 3)
1k8d_P_A.txt
(48, 3, 3)
1k9q_A.txt
(5, 3, 3)
1k9q_B.txt
(10, 3, 3)
1k9r_A.txt
(5, 3, 3)
1k9r_B.txt
(10, 3, 3)
1ka6_B_A.txt
(22, 3, 3)
1ka7_B_A.txt
(38, 3, 3)
1kap_I_P.txt
(25, 3, 3)
1kat_X_V.txt
(24, 3, 3)
1kat_Y_V.txt
(23, 3, 3)
1kbg_P_H.txt
(43, 3, 3)
1kbh_A.txt
(39, 3, 3)
1kc2_B_A.txt
(15, 3, 3)
1kc4_B_A.txt
(26, 3, 3)
1kc5_P_H.txt
(26, 3, 3)
1kcr_P.txt
(25, 3, 3)
1kcs_P_H.txt
(24, 3, 3)
1kd8_A.txt
(20, 3, 3)
1kd8_B.txt
(19, 3, 3)
1kd9_A.txt
(20, 3, 3)
1kd9_B.txt
(20, 3, 3)
1kdd_A.txt
(19, 3, 3)
1kdd_B.txt
(20, 3, 3)
1kdv_B.txt
(10, 3, 3)
1kdx_B_A.txt
(29, 3, 3)
1kdy_B.txt
(10, 3, 3)
1kdz_B.txt
(13, 3, 3)
1kg0_D.txt
(44, 3, 3)
1khp_I_A.txt
(22, 3, 3)
1khq_I_A.txt
(21, 3, 3)
1kil_E_A.txt
(22, 3, 3)
1kj2_P.txt
(44, 3, 3)
1kj3_P_H.txt
(44, 3, 3)
1kj3_Q_I.txt
(45, 3, 3)
1kj4_P.txt
(38, 3, 3)
1kj7_P_A.txt
(42, 3, 3)
1kjf_P_A.txt
(43, 3, 3)
1kjg_P_A.txt
(38, 3, 3)
1kjh_P_A.txt
(43, 3, 3)
1kjm_P_A.txt
(43, 3, 3)
1kjv_P_A

1sn9_A.txt
(16, 3, 3)
1sna_A.txt
(2, 3, 3)
1sne_A.txt
(6, 3, 3)
1soz_D_A.txt
(24, 3, 3)
1soz_E_B.txt
(24, 3, 3)
1sp4_A_B.txt
(91, 3, 3)
1sp5_I_A.txt
(28, 3, 3)
1sps_D.txt
(13, 3, 3)
1sqk_B_A.txt
(29, 3, 3)
1sqz_B_A.txt
(13, 3, 3)
1sr5_B.txt
(40, 3, 3)
1srn_B_A.txt
(38, 3, 3)
1ssa_B_A.txt
(36, 3, 3)
1ssb_B_A.txt
(34, 3, 3)
1ssc_B_A.txt
(37, 3, 3)
1sse_A_B.txt
(21, 3, 3)
1ssh_B_A.txt
(16, 3, 3)
1stc_I_E.txt
(28, 3, 3)
1str_M_B.txt
(23, 3, 3)
1str_P_D.txt
(25, 3, 3)
1sts_M_B.txt
(25, 3, 3)
1sts_P_D.txt
(26, 3, 3)
1sua_C_A.txt
(29, 3, 3)
1suy_C_A.txt
(78, 3, 3)
1suy_D_A.txt
(78, 3, 3)
1sv1_C_A.txt
(78, 3, 3)
1sv1_D_A.txt
(76, 3, 3)
1sve_B_A.txt
(38, 3, 3)
1svf_B_A.txt
(30, 3, 3)
1svf_D_C.txt
(31, 3, 3)
1svg_B_A.txt
(43, 3, 3)
1svh_B_A.txt
(38, 3, 3)
1svz_C_A.txt
(32, 3, 3)
1svz_D_B.txt
(31, 3, 3)
1swi_A.txt
(25, 3, 3)
1sy9_B_A.txt
(62, 3, 3)
1syq_B_A.txt
(41, 3, 3)
1sys_C_A.txt
(52, 3, 3)
1syv_C_A.txt
(48, 3, 3)
1sza_Z_B.txt
(17, 3, 3)
1szc_B_A.txt
(17, 3, 3)
1szd_B_A.txt
(18, 3, 3)
1t01_B

2dvq_P_A.txt
(16, 3, 3)
2dvq_Q_A.txt
(37, 3, 3)
2dvr_P_A.txt
(16, 3, 3)
2dvr_Q_A.txt
(25, 3, 3)
2dwv_A.txt
(16, 3, 3)
2dwx_P.txt
(22, 3, 3)
2dxp_B_A.txt
(4, 3, 3)
2dyf_A.txt
(6, 3, 3)
2dyf_B.txt
(8, 3, 3)
2dyh_B_A.txt
(27, 3, 3)
2dym_B_A.txt
(55, 3, 3)
2dym_D_C.txt
(53, 3, 3)
2dym_F_A.txt
(54, 3, 3)
2dym_H_C.txt
(57, 3, 3)
2dyo_B_A.txt
(57, 3, 3)
2dyp_C_A.txt
(44, 3, 3)
2dyr_K_A.txt
(62, 3, 3)
2dyr_L_A.txt
(55, 3, 3)
2dyr_M_A.txt
(48, 3, 3)
2dyr_X_N.txt
(62, 3, 3)
2dyr_Y_N.txt
(56, 3, 3)
2dyr_Z_N.txt
(48, 3, 3)
2dys_K_A.txt
(65, 3, 3)
2dys_L_A.txt
(55, 3, 3)
2dys_M_A.txt
(46, 3, 3)
2dys_X_N.txt
(62, 3, 3)
2dys_Y_N.txt
(55, 3, 3)
2dys_Z_N.txt
(46, 3, 3)
2dze_X_A.txt
(35, 3, 3)
2e30_B_A.txt
(79, 3, 3)
2e3k_Q_B.txt
(25, 3, 3)
2e3k_R_D.txt
(6, 3, 3)
2e4h_B_A.txt
(22, 3, 3)
2e74_E.txt
(32, 3, 3)
2e74_F.txt
(42, 3, 3)
2e74_G.txt
(57, 3, 3)
2e74_H.txt
(68, 3, 3)
2e75_E.txt
(33, 3, 3)
2e75_F.txt
(40, 3, 3)
2e75_G.txt
(55, 3, 3)
2e75_H.txt
(68, 3, 3)
2e76_E.txt
(34, 3, 3)
2e76_F.txt
(40, 3, 3)


2occ_Z_N.txt
(47, 3, 3)
2ocv_A_B.txt
(53, 3, 3)
2od2_B_A.txt
(17, 3, 3)
2od3_A_B.txt
(40, 3, 3)
2od7_B_A.txt
(18, 3, 3)
2od8_B.txt
(23, 3, 3)
2od9_B_A.txt
(17, 3, 3)
2odb_B_A.txt
(39, 3, 3)
2odd_B.txt
(16, 3, 3)
2odg_C_A.txt
(26, 3, 3)
2ody_A_B.txt
(63, 3, 3)
2ody_C_B.txt
(56, 3, 3)
2oei_A.txt
(8, 3, 3)
2oei_B.txt
(10, 3, 3)
2ofq_B_A.txt
(36, 3, 3)
2oh0_I_E.txt
(40, 3, 3)
2oi3_B_A.txt
(27, 3, 3)
2oi9_Q_A.txt
(61, 3, 3)
2oin_C_A.txt
(43, 3, 3)
2oin_D_A.txt
(69, 3, 3)
2oj2_B_A.txt
(26, 3, 3)
2oje_C.txt
(50, 3, 3)
2ojf_I_E.txt
(43, 3, 3)
2oju_C_A.txt
(1, 3, 3)
2oju_D_B.txt
(1, 3, 3)
2ojx_E.txt
(9, 3, 3)
2okr_C_A.txt
(30, 3, 3)
2okr_F_A.txt
(38, 3, 3)
2okz_A.txt
(3, 3, 3)
2ol3_P.txt
(42, 3, 3)
2olb_B_A.txt
(39, 3, 3)
2oly_A.txt
(25, 3, 3)
2oly_B.txt
(27, 3, 3)
2olz_A.txt
(25, 3, 3)
2olz_B.txt
(28, 3, 3)
2om0_A.txt
(25, 3, 3)
2om0_B.txt
(35, 3, 3)
2om1_A.txt
(25, 3, 3)
2om1_B.txt
(35, 3, 3)
2om2_B_A.txt
(73, 3, 3)
2om2_D_C.txt
(69, 3, 3)
2omg_A.txt
(18, 3, 3)
2omg_B.txt
(23, 3, 3)
2omh_A.tx

...

5kd7_E_C.txt
(47, 3, 3)
5kd7_H_F.txt
(46, 3, 3)
5kd7_K_C.txt
(45, 3, 3)
5kd7_P_A.txt
(45, 3, 3)
5kd9_C_A.txt
(18, 3, 3)
5kd9_D_B.txt
(17, 3, 3)
5kds_G_A.txt
(12, 3, 3)
5kez_B_A.txt
(31, 3, 3)
5kgf_L.txt
(24, 3, 3)
5kgn_C_A.txt
(37, 3, 3)
5kgn_D_B.txt
(37, 3, 3)
5kht_A.txt
(28, 3, 3)
5kjh_D_A.txt
(24, 3, 3)
5kjr_N.txt
(18, 3, 3)
5kk3_A.txt
(41, 3, 3)
5kk4_A.txt
(9, 3, 3)
5kkl_D.txt
(18, 3, 3)
5klr_A_B.txt
(49, 3, 3)
5klt_A.txt
(28, 3, 3)
5kmx_U.txt
(15, 3, 3)
5kn5_C.txt
(26, 3, 3)
5kna_A.txt
(29, 3, 3)
5knm_N.txt
(22, 3, 3)
5ko0_A.txt
(11, 3, 3)
5koa_D.txt
(12, 3, 3)
5kog_P.txt
(17, 3, 3)
5kq1_C.txt
(24, 3, 3)
5kq4_F.txt
(24, 3, 3)
5kqv_A.txt
(12, 3, 3)
5kqv_B.txt
(11, 3, 3)
5kr9_C_A.txt
(19, 3, 3)
5kr9_D_B.txt
(17, 3, 3)
5kra_C_A.txt
(18, 3, 3)
5kra_D_B.txt
(17, 3, 3)
5kra_G_E.txt
(18, 3, 3)
5kra_H_F.txt
(19, 3, 3)
5krc_C_B.txt
(18, 3, 3)
5krc_D_A.txt
(18, 3, 3)
5krf_C_A.txt
(18, 3, 3)
5krf_D_B.txt
(19, 3, 3)
5krh_C_A.txt
(19, 3, 3)
5krh_D_B.txt
(17, 3, 3)
5kri_C_A.txt
(18, 3, 3)
5kri_

5ts1_P_A.txt
(52, 3, 3)
5ts1_Q_C.txt
(51, 3, 3)
5ts1_R_E.txt
(51, 3, 3)
5ts1_S_G.txt
(49, 3, 3)
5tsv_D_A.txt
(48, 3, 3)
5tsx_M_A.txt
(35, 3, 3)
5tsx_N_B.txt
(52, 3, 3)
5tsx_O_J.txt
(50, 3, 3)
5tsx_P_I.txt
(49, 3, 3)
5tsx_R_E.txt
(26, 3, 3)
5tsx_T_F.txt
(51, 3, 3)
5ttw_B_A.txt
(10, 3, 3)
5ttw_D_C.txt
(10, 3, 3)
5tu5_B_A.txt
(19, 3, 3)
5tu6_B_A.txt
(26, 3, 3)
5tuv_C.txt
(31, 3, 3)
5tw1_G.txt
(10, 3, 3)
5twa_C_B.txt
(42, 3, 3)
5twa_D_A.txt
(40, 3, 3)
5twg_E_A.txt
(11, 3, 3)
5twh_E_A.txt
(13, 3, 3)
5two_B_A.txt
(21, 3, 3)
5txe_C_A.txt
(49, 3, 3)
5txe_D_B.txt
(51, 3, 3)
5txh_A.txt
(7, 3, 3)
5txj_A.txt
(6, 3, 3)
5txs_C_A.txt
(57, 3, 3)
5tyi_L_A.txt
(17, 3, 3)
5tyi_M_B.txt
(18, 3, 3)
5tyi_N_A.txt
(20, 3, 3)
5tyi_P_B.txt
(18, 3, 3)
5tzp_B_A.txt
(37, 3, 3)
5tzq_C_B.txt
(38, 3, 3)
5tzq_D_A.txt
(40, 3, 3)
5u06_L_A.txt
(15, 3, 3)
5u06_M_B.txt
(19, 3, 3)
5u06_N_B.txt
(14, 3, 3)
5u06_P_A.txt
(18, 3, 3)
5u1g_K.txt
(29, 3, 3)
5u1g_Z.txt
(16, 3, 3)
5u1m_B_A.txt
(20, 3, 3)
5u1q_L_A.txt
(20, 3, 3)
5u1q_M

5zvk_b.txt
(44, 3, 3)
5zvm_a.txt
(47, 3, 3)
5zvm_b.txt
(54, 3, 3)
5zvm_c.txt
(52, 3, 3)
5zvt_A.txt
(59, 3, 3)
5zvw_B_A.txt
(39, 3, 3)
5zw6_C_A.txt
(33, 3, 3)
5zw6_D_B.txt
(32, 3, 3)
5zwe_C.txt
(16, 3, 3)
5zwf_C_A.txt
(19, 3, 3)
5zwh_C_A.txt
(17, 3, 3)
5zwi_C_A.txt
(19, 3, 3)
5zwn_x.txt
(3, 3, 3)
5zwx_P_A.txt
(26, 3, 3)
5zwx_Q_A.txt
(29, 3, 3)
5zys_B_A.txt
(20, 3, 3)
5zz9_E.txt
(22, 3, 3)
5zzn_f.txt
(36, 3, 3)
5zzn_j.txt
(30, 3, 3)
5zzn_K.txt
(23, 3, 3)
5zzn_l.txt
(47, 3, 3)
5zzn_m.txt
(9, 3, 3)
5zzn_T.txt
(33, 3, 3)
5zzn_x.txt
(17, 3, 3)
5zzn_Y.txt
(24, 3, 3)
6a0a_A.txt
(47, 3, 3)
6a0a_B.txt
(47, 3, 3)
6a0c_A.txt
(46, 3, 3)
6a0c_B.txt
(47, 3, 3)
6a0f_C_A.txt
(29, 3, 3)
6a0f_D_B.txt
(28, 3, 3)
6a0h_C.txt
(22, 3, 3)
6a22_B.txt
(20, 3, 3)
6a2b_C.txt
(39, 3, 3)
6a30_P.txt
(11, 3, 3)
6a33_I_A.txt
(23, 3, 3)
6a38_D.txt
(27, 3, 3)
6a3a_D_C.txt
(32, 3, 3)
6a3b_D.txt
(27, 3, 3)
6a3c_D_C.txt
(32, 3, 3)
6a3e_D.txt
(28, 3, 3)
6a5e_E.txt
(41, 3, 3)
6a5q_D_C.txt
(39, 3, 3)
6a5q_E_B.txt
(39, 3, 3)
6a

6i68_D_C.txt
(17, 3, 3)
6i68_F_E.txt
(17, 3, 3)
6i68_H_G.txt
(14, 3, 3)
6i6h_B_A.txt
(32, 3, 3)
6i7a_B_A.txt
(14, 3, 3)
6i7a_D_C.txt
(15, 3, 3)
6i7a_F_E.txt
(13, 3, 3)
6i7a_H_G.txt
(16, 3, 3)
6i7b_B.txt
(53, 3, 3)
6i7m_B.txt
(53, 3, 3)
6i7q_H_V.txt
(34, 3, 3)
6i7r_H_V.txt
(35, 3, 3)
6i85_B.txt
(61, 3, 3)
6iam_B_A.txt
(44, 3, 3)
6iam_C_A.txt
(6, 3, 3)
6iaw_X.txt
(11, 3, 3)
6iaw_Y.txt
(17, 3, 3)
6iaw_Z.txt
(12, 3, 3)
6ibo_C_A.txt
(18, 3, 3)
6icj_B_A.txt
(19, 3, 3)
6ict_E_A.txt
(50, 3, 3)
6ict_G_B.txt
(46, 3, 3)
6ict_H_C.txt
(51, 3, 3)
6ict_I_D.txt
(48, 3, 3)
6icv_C_A.txt
(54, 3, 3)
6icv_D_B.txt
(51, 3, 3)
6id4_T_E.txt
(45, 3, 3)
6id4_U_A.txt
(46, 3, 3)
6idx_C_A.txt
(57, 3, 3)
6ie4_B.txt
(19, 3, 3)
6ie6_P_A.txt
(24, 3, 3)
6ie7_E.txt
(18, 3, 3)
6ieu_C_A.txt
(29, 3, 3)
6iex_C_A.txt
(43, 3, 3)
6ifc_B_A.txt
(53, 3, 3)
6ifc_D_A.txt
(52, 3, 3)
6ifc_F_E.txt
(53, 3, 3)
6ifc_H_E.txt
(54, 3, 3)
6ifg_E_A.txt
(24, 3, 3)
6ifg_F_B.txt
(27, 3, 3)
6ifj_C_A.txt
(23, 3, 3)
6ifj_D_B.txt
(22, 3, 3)
6igk_B_A.

6rqf_M.txt
(35, 3, 3)
6rqf_N.txt
(37, 3, 3)
6rqf_O.txt
(43, 3, 3)
6rqf_P.txt
(26, 3, 3)
6rqx_B_A.txt
(51, 3, 3)
6rr0_H_A.txt
(21, 3, 3)
6rr0_I_A.txt
(22, 3, 3)
6rr0_J_E.txt
(16, 3, 3)
6rr0_K_B.txt
(15, 3, 3)
6rr0_L_C.txt
(22, 3, 3)
6rr0_M_F.txt
(21, 3, 3)
6rr0_N_G.txt
(11, 3, 3)
6rrc_B_C.txt
(32, 3, 3)
6rrc_D_A.txt
(37, 3, 3)
6rrk_D.txt
(43, 3, 3)
6rup_C_A.txt
(14, 3, 3)
6rw2_B_A.txt
(34, 3, 3)
6rx4_C.txt
(35, 3, 3)
6rx4_D.txt
(19, 3, 3)
6rxh_A_B.txt
(75, 3, 3)
6rxh_D_B.txt
(64, 3, 3)
6rxj_C_A.txt
(23, 3, 3)
6rxj_D_B.txt
(28, 3, 3)
6rxk_B_A.txt
(27, 3, 3)
6rxl_B_A.txt
(13, 3, 3)
6rxm_G_A.txt
(25, 3, 3)
6rxm_H_A.txt
(25, 3, 3)
6rxm_I_C.txt
(26, 3, 3)
6rxm_J_C.txt
(26, 3, 3)
6rxm_K_C.txt
(33, 3, 3)
6rxm_L_E.txt
(32, 3, 3)
6rxo_C_A.txt
(29, 3, 3)
6rxo_D_A.txt
(24, 3, 3)
6rxp_C_A.txt
(29, 3, 3)
6rxp_D_A.txt
(40, 3, 3)
6rxq_E_A.txt
(31, 3, 3)
6rxq_F_A.txt
(34, 3, 3)
6rxq_G_C.txt
(18, 3, 3)
6rxq_H_D.txt
(18, 3, 3)
6rxr_E_A.txt
(31, 3, 3)
6rxr_F_A.txt
(35, 3, 3)
6rxr_G_C.txt
(18, 3, 3)
6rxr_H

# Part of the gvp data, name and corresponding coordinates

In [47]:
#判断表示=链名+氨基酸序列号
with jsonlines.open('./gvp_json/gvp6A_primary.jsonl', mode='w') as writer:  
    path = ['./pocket6A_gvp']
    save_name =[]
    for p in path:
        now_num=0
        all_num=len(os.listdir(p))
        for pdb_file in os.listdir(p):
            now_num=now_num+1
#             print("\r", end="")
#             print("Download progress: {}% ".format(now_num/all_num*100), end="")
#             sys.stdout.flush()
            count_number = 0
            resID=[]
            resList=[]
            resCoors = []
            pdb_data = {}
            resCoor =[]
            flag = 1
    #         xxx=1
    #         yyy = 1
            with open(os.path.join(p,pdb_file)) as f:
                lines= f.readline()
                line = lines.strip()
                #line_split = line.split()

                while line:
                    x = name_to_simple(line[17:20])
                    if x !=None and line[0:4] =="ATOM":
                        if line[21:26] not in resID:
                            resList.append(x)
    #                         if xxx!=yyy:
    #                             print(yyy)
    #                         xxx +=4
                            resID.append(line[21:26])

                        rescoor = []
                        x1,y,z =float(line[30:38]),float(line[38:46]),float(line[46:54])
                        rescoor.append(x1)
                        rescoor.append(y)
                        rescoor.append(z)
                       

                        count_number+=1
                        resCoor.append(rescoor)
                        if count_number==4:
                            resCoors.append(resCoor)
                            count_number =0 
                            resCoor = []
    #                 if len(resList)!=len(resCoors):
    #                     print(len(resList))
    #                     print(len(resCoors))
                    lines= f.readline()
                    line = lines.strip()
    #                yyy+=1
                    #line_split = line.split()
            pdb_data['name'] = pdb_file
            pdb_data_seq = "".join(resList)
            pdb_data['coords'] = resCoors
            if len(pdb_data_seq)==len(resCoors) and len(pdb_data_seq)!=0:
                writer.write(pdb_data)
            else:
                with open("./eooro.txt","a+") as errorf:
                    errorf.write("\n")
                    errorf.write(pdb_file)
#                 print("error in length")
#                 print(pdb_data['name'])
#                 print(len(pdb_data_seq))
#                 print(len(resCoors))

In [48]:
num = 0
with open("./gvp_json/gvp6A_primary.jsonl", "r+", encoding="utf8") as f:
    for item in jsonlines.Reader(f):
        num+=1
        #print(item["name"])
        print(np.array(item["coords"]).shape)

(9, 4, 3)
(6, 4, 3)
(6, 4, 3)
(7, 4, 3)
(5, 4, 3)
(6, 4, 3)
(7, 4, 3)
(6, 4, 3)
(11, 4, 3)
(7, 4, 3)
(7, 4, 3)
(6, 4, 3)
(6, 4, 3)
(6, 4, 3)
(6, 4, 3)
(6, 4, 3)
(5, 4, 3)
(41, 4, 3)
(41, 4, 3)
(47, 4, 3)
(46, 4, 3)
(72, 4, 3)
(19, 4, 3)
(16, 4, 3)
(7, 4, 3)
(50, 4, 3)
(40, 4, 3)
(25, 4, 3)
(10, 4, 3)
(20, 4, 3)
(15, 4, 3)
(39, 4, 3)
(17, 4, 3)
(40, 4, 3)
(12, 4, 3)
(14, 4, 3)
(12, 4, 3)
(14, 4, 3)
(39, 4, 3)
(15, 4, 3)
(37, 4, 3)
(15, 4, 3)
(38, 4, 3)
(17, 4, 3)
(38, 4, 3)
(16, 4, 3)
(16, 4, 3)
(38, 4, 3)
(47, 4, 3)
(33, 4, 3)
(26, 4, 3)
(16, 4, 3)
(14, 4, 3)
(28, 4, 3)
(17, 4, 3)
(15, 4, 3)
(33, 4, 3)
(42, 4, 3)
(26, 4, 3)
(26, 4, 3)
(34, 4, 3)
(45, 4, 3)
(40, 4, 3)
(13, 4, 3)
(31, 4, 3)
(19, 4, 3)
(17, 4, 3)
(41, 4, 3)
(17, 4, 3)
(51, 4, 3)
(17, 4, 3)
(52, 4, 3)
(12, 4, 3)
(27, 4, 3)
(41, 4, 3)
(42, 4, 3)
(41, 4, 3)
(43, 4, 3)
(51, 4, 3)
(17, 4, 3)
(43, 4, 3)
(15, 4, 3)
(15, 4, 3)
(21, 4, 3)
(16, 4, 3)
(39, 4, 3)
(20, 4, 3)
(16, 4, 3)
(19, 4, 3)
(39, 4, 3)
(19, 4, 3)
(17, 4, 3)
(17, 

(21, 4, 3)
(26, 4, 3)
(33, 4, 3)
(24, 4, 3)
(24, 4, 3)
(16, 4, 3)
(25, 4, 3)
(25, 4, 3)
(16, 4, 3)
(17, 4, 3)
(44, 4, 3)
(44, 4, 3)
(26, 4, 3)
(39, 4, 3)
(33, 4, 3)
(42, 4, 3)
(43, 4, 3)
(125, 4, 3)
(126, 4, 3)
(124, 4, 3)
(124, 4, 3)
(20, 4, 3)
(20, 4, 3)
(21, 4, 3)
(23, 4, 3)
(22, 4, 3)
(21, 4, 3)
(34, 4, 3)
(28, 4, 3)
(34, 4, 3)
(28, 4, 3)
(16, 4, 3)
(39, 4, 3)
(16, 4, 3)
(39, 4, 3)
(16, 4, 3)
(40, 4, 3)
(15, 4, 3)
(39, 4, 3)
(17, 4, 3)
(37, 4, 3)
(23, 4, 3)
(30, 4, 3)
(28, 4, 3)
(41, 4, 3)
(31, 4, 3)
(17, 4, 3)
(13, 4, 3)
(18, 4, 3)
(16, 4, 3)
(28, 4, 3)
(20, 4, 3)
(21, 4, 3)
(44, 4, 3)
(124, 4, 3)
(131, 4, 3)
(122, 4, 3)
(134, 4, 3)
(124, 4, 3)
(130, 4, 3)
(126, 4, 3)
(131, 4, 3)
(125, 4, 3)
(133, 4, 3)
(125, 4, 3)
(132, 4, 3)
(126, 4, 3)
(131, 4, 3)
(126, 4, 3)
(132, 4, 3)
(126, 4, 3)
(129, 4, 3)
(126, 4, 3)
(125, 4, 3)
(14, 4, 3)
(58, 4, 3)
(59, 4, 3)
(20, 4, 3)
(17, 4, 3)
(83, 4, 3)
(90, 4, 3)
(83, 4, 3)
(89, 4, 3)
(31, 4, 3)
(14, 4, 3)
(14, 4, 3)
(20, 4, 3)
(107, 4, 3)
(81, 4,

(18, 4, 3)
(19, 4, 3)
(23, 4, 3)
(17, 4, 3)
(18, 4, 3)
(15, 4, 3)
(14, 4, 3)
(15, 4, 3)
(24, 4, 3)
(30, 4, 3)
(22, 4, 3)
(18, 4, 3)
(20, 4, 3)
(19, 4, 3)
(18, 4, 3)
(18, 4, 3)
(18, 4, 3)
(14, 4, 3)
(16, 4, 3)
(11, 4, 3)
(17, 4, 3)
(32, 4, 3)
(27, 4, 3)
(27, 4, 3)
(25, 4, 3)
(49, 4, 3)
(51, 4, 3)
(50, 4, 3)
(22, 4, 3)
(20, 4, 3)
(23, 4, 3)
(45, 4, 3)
(46, 4, 3)
(45, 4, 3)
(47, 4, 3)
(18, 4, 3)
(45, 4, 3)
(46, 4, 3)
(46, 4, 3)
(46, 4, 3)
(27, 4, 3)
(27, 4, 3)
(36, 4, 3)
(57, 4, 3)
(58, 4, 3)
(14, 4, 3)
(15, 4, 3)
(13, 4, 3)
(17, 4, 3)
(18, 4, 3)
(33, 4, 3)
(29, 4, 3)
(8, 4, 3)
(26, 4, 3)
(18, 4, 3)
(17, 4, 3)
(17, 4, 3)
(51, 4, 3)
(18, 4, 3)
(15, 4, 3)
(5, 4, 3)
(16, 4, 3)
(17, 4, 3)
(14, 4, 3)
(17, 4, 3)
(17, 4, 3)
(14, 4, 3)
(13, 4, 3)
(18, 4, 3)
(16, 4, 3)
(19, 4, 3)
(17, 4, 3)
(17, 4, 3)
(17, 4, 3)
(21, 4, 3)
(21, 4, 3)
(21, 4, 3)
(13, 4, 3)
(30, 4, 3)
(24, 4, 3)
(12, 4, 3)
(6, 4, 3)
(35, 4, 3)
(32, 4, 3)
(16, 4, 3)
(59, 4, 3)
(29, 4, 3)
(38, 4, 3)
(34, 4, 3)
(22, 4, 3)
(22, 4, 3)
(5

(47, 4, 3)
(49, 4, 3)
(39, 4, 3)
(48, 4, 3)
(45, 4, 3)
(19, 4, 3)
(18, 4, 3)
(51, 4, 3)
(49, 4, 3)
(49, 4, 3)
(49, 4, 3)
(21, 4, 3)
(19, 4, 3)
(19, 4, 3)
(19, 4, 3)
(19, 4, 3)
(24, 4, 3)
(18, 4, 3)
(18, 4, 3)
(19, 4, 3)
(32, 4, 3)
(18, 4, 3)
(13, 4, 3)
(7, 4, 3)
(27, 4, 3)
(30, 4, 3)
(21, 4, 3)
(58, 4, 3)
(26, 4, 3)
(26, 4, 3)
(13, 4, 3)
(27, 4, 3)
(29, 4, 3)
(26, 4, 3)
(35, 4, 3)
(16, 4, 3)
(37, 4, 3)
(22, 4, 3)
(21, 4, 3)
(43, 4, 3)
(33, 4, 3)
(30, 4, 3)
(52, 4, 3)
(17, 4, 3)
(30, 4, 3)
(18, 4, 3)
(28, 4, 3)
(41, 4, 3)
(41, 4, 3)
(45, 4, 3)
(39, 4, 3)
(16, 4, 3)
(27, 4, 3)
(23, 4, 3)
(44, 4, 3)
(41, 4, 3)
(47, 4, 3)
(45, 4, 3)
(43, 4, 3)
(35, 4, 3)
(29, 4, 3)
(45, 4, 3)
(32, 4, 3)
(18, 4, 3)
(13, 4, 3)
(13, 4, 3)
(28, 4, 3)
(32, 4, 3)
(18, 4, 3)
(14, 4, 3)
(20, 4, 3)
(41, 4, 3)
(42, 4, 3)
(21, 4, 3)
(37, 4, 3)
(36, 4, 3)
(41, 4, 3)
(41, 4, 3)
(42, 4, 3)
(40, 4, 3)
(39, 4, 3)
(18, 4, 3)
(74, 4, 3)
(45, 4, 3)
(31, 4, 3)
(23, 4, 3)
(22, 4, 3)
(30, 4, 3)
(37, 4, 3)
(29, 4, 3)
(19, 4, 3)


(33, 4, 3)
(35, 4, 3)
(31, 4, 3)
(40, 4, 3)
(6, 4, 3)
(20, 4, 3)
(91, 4, 3)
(90, 4, 3)
(24, 4, 3)
(14, 4, 3)
(18, 4, 3)
(15, 4, 3)
(50, 4, 3)
(50, 4, 3)
(42, 4, 3)
(48, 4, 3)
(49, 4, 3)
(45, 4, 3)
(44, 4, 3)
(45, 4, 3)
(47, 4, 3)
(40, 4, 3)
(47, 4, 3)
(44, 4, 3)
(37, 4, 3)
(43, 4, 3)
(29, 4, 3)
(24, 4, 3)
(7, 4, 3)
(7, 4, 3)
(9, 4, 3)
(8, 4, 3)
(8, 4, 3)
(7, 4, 3)
(4, 4, 3)
(7, 4, 3)
(5, 4, 3)
(7, 4, 3)
(6, 4, 3)
(6, 4, 3)
(5, 4, 3)
(9, 4, 3)
(7, 4, 3)
(8, 4, 3)
(6, 4, 3)
(5, 4, 3)
(6, 4, 3)
(7, 4, 3)
(4, 4, 3)
(6, 4, 3)
(6, 4, 3)
(7, 4, 3)
(3, 4, 3)
(4, 4, 3)
(3, 4, 3)
(3, 4, 3)
(20, 4, 3)
(31, 4, 3)
(20, 4, 3)
(18, 4, 3)
(23, 4, 3)
(25, 4, 3)
(22, 4, 3)
(46, 4, 3)
(47, 4, 3)
(22, 4, 3)
(21, 4, 3)
(47, 4, 3)
(17, 4, 3)
(31, 4, 3)
(35, 4, 3)
(35, 4, 3)
(30, 4, 3)
(32, 4, 3)
(29, 4, 3)
(27, 4, 3)
(43, 4, 3)
(14, 4, 3)
(28, 4, 3)
(39, 4, 3)
(26, 4, 3)
(32, 4, 3)
(38, 4, 3)
(44, 4, 3)
(16, 4, 3)
(49, 4, 3)
(12, 4, 3)
(62, 4, 3)
(32, 4, 3)
(38, 4, 3)
(34, 4, 3)
(14, 4, 3)
(11, 4, 3)
(39, 4

...

(42, 4, 3)
(44, 4, 3)
(1, 4, 3)
(10, 4, 3)
(9, 4, 3)
(27, 4, 3)
(19, 4, 3)
(25, 4, 3)
(15, 4, 3)
(15, 4, 3)
(14, 4, 3)
(12, 4, 3)
(22, 4, 3)
(22, 4, 3)
(20, 4, 3)
(15, 4, 3)
(26, 4, 3)
(26, 4, 3)
(16, 4, 3)
(39, 4, 3)
(37, 4, 3)
(10, 4, 3)
(21, 4, 3)
(15, 4, 3)
(14, 4, 3)
(14, 4, 3)
(15, 4, 3)
(10, 4, 3)
(10, 4, 3)
(15, 4, 3)
(32, 4, 3)
(18, 4, 3)
(18, 4, 3)
(17, 4, 3)
(16, 4, 3)
(54, 4, 3)
(43, 4, 3)
(17, 4, 3)
(16, 4, 3)
(22, 4, 3)
(29, 4, 3)
(11, 4, 3)
(1, 4, 3)
(1, 4, 3)
(2, 4, 3)
(2, 4, 3)
(26, 4, 3)
(15, 4, 3)
(27, 4, 3)
(31, 4, 3)
(38, 4, 3)
(33, 4, 3)
(30, 4, 3)
(43, 4, 3)
(31, 4, 3)
(25, 4, 3)
(28, 4, 3)
(24, 4, 3)
(47, 4, 3)
(43, 4, 3)
(22, 4, 3)
(20, 4, 3)
(41, 4, 3)
(40, 4, 3)
(16, 4, 3)
(15, 4, 3)
(26, 4, 3)
(29, 4, 3)
(28, 4, 3)
(27, 4, 3)
(13, 4, 3)
(20, 4, 3)
(5, 4, 3)
(3, 4, 3)
(1, 4, 3)
(2, 4, 3)
(5, 4, 3)
(1, 4, 3)
(5, 4, 3)
(25, 4, 3)
(20, 4, 3)
(33, 4, 3)
(36, 4, 3)
(53, 4, 3)
(1, 4, 3)
(2, 4, 3)
(10, 4, 3)
(3, 4, 3)
(19, 4, 3)
(20, 4, 3)
(23, 4, 3)
(19, 4, 3)
(7, 

(18, 4, 3)
(19, 4, 3)
(41, 4, 3)
(47, 4, 3)
(48, 4, 3)
(49, 4, 3)
(42, 4, 3)
(46, 4, 3)
(43, 4, 3)
(3, 4, 3)
(55, 4, 3)
(55, 4, 3)
(45, 4, 3)
(27, 4, 3)
(30, 4, 3)
(16, 4, 3)
(17, 4, 3)
(23, 4, 3)
(98, 4, 3)
(100, 4, 3)
(97, 4, 3)
(102, 4, 3)
(97, 4, 3)
(103, 4, 3)
(97, 4, 3)
(100, 4, 3)
(21, 4, 3)
(22, 4, 3)
(22, 4, 3)
(22, 4, 3)
(38, 4, 3)
(21, 4, 3)
(22, 4, 3)
(22, 4, 3)
(23, 4, 3)
(22, 4, 3)
(19, 4, 3)
(24, 4, 3)
(17, 4, 3)
(17, 4, 3)
(17, 4, 3)
(7, 4, 3)
(96, 4, 3)
(103, 4, 3)
(98, 4, 3)
(104, 4, 3)
(96, 4, 3)
(103, 4, 3)
(96, 4, 3)
(104, 4, 3)
(64, 4, 3)
(63, 4, 3)
(66, 4, 3)
(52, 4, 3)
(53, 4, 3)
(35, 4, 3)
(21, 4, 3)
(29, 4, 3)
(97, 4, 3)
(104, 4, 3)
(97, 4, 3)
(104, 4, 3)
(97, 4, 3)
(104, 4, 3)
(97, 4, 3)
(102, 4, 3)
(28, 4, 3)
(28, 4, 3)
(19, 4, 3)
(51, 4, 3)
(18, 4, 3)
(2, 4, 3)
(16, 4, 3)
(15, 4, 3)
(3, 4, 3)
(17, 4, 3)
(98, 4, 3)
(102, 4, 3)
(21, 4, 3)
(21, 4, 3)
(47, 4, 3)
(2, 4, 3)
(13, 4, 3)
(67, 4, 3)
(70, 4, 3)
(70, 4, 3)
(66, 4, 3)
(55, 4, 3)
(56, 4, 3)
(31, 4, 3)
(2

(57, 4, 3)
(50, 4, 3)
(63, 4, 3)
(57, 4, 3)
(48, 4, 3)
(63, 4, 3)
(58, 4, 3)
(46, 4, 3)
(22, 4, 3)
(27, 4, 3)
(6, 4, 3)
(9, 4, 3)
(29, 4, 3)
(20, 4, 3)
(4, 4, 3)
(25, 4, 3)
(107, 4, 3)
(108, 4, 3)
(32, 4, 3)
(28, 4, 3)
(31, 4, 3)
(31, 4, 3)
(28, 4, 3)
(31, 4, 3)
(24, 4, 3)
(49, 4, 3)
(38, 4, 3)
(24, 4, 3)
(25, 4, 3)
(17, 4, 3)
(11, 4, 3)
(14, 4, 3)
(13, 4, 3)
(9, 4, 3)
(9, 4, 3)
(35, 4, 3)
(13, 4, 3)
(13, 4, 3)
(14, 4, 3)
(15, 4, 3)
(33, 4, 3)
(19, 4, 3)
(21, 4, 3)
(23, 4, 3)
(8, 4, 3)
(8, 4, 3)
(33, 4, 3)
(31, 4, 3)
(41, 4, 3)
(44, 4, 3)
(48, 4, 3)
(17, 4, 3)
(18, 4, 3)
(21, 4, 3)
(17, 4, 3)
(13, 4, 3)
(32, 4, 3)
(32, 4, 3)
(44, 4, 3)
(25, 4, 3)
(53, 4, 3)
(39, 4, 3)
(29, 4, 3)
(45, 4, 3)
(29, 4, 3)
(53, 4, 3)
(8, 4, 3)
(31, 4, 3)
(20, 4, 3)
(15, 4, 3)
(44, 4, 3)
(34, 4, 3)
(40, 4, 3)
(26, 4, 3)
(24, 4, 3)
(27, 4, 3)
(28, 4, 3)
(27, 4, 3)
(27, 4, 3)
(27, 4, 3)
(26, 4, 3)
(28, 4, 3)
(31, 4, 3)
(39, 4, 3)
(41, 4, 3)
(35, 4, 3)
(21, 4, 3)
(27, 4, 3)
(28, 4, 3)
(33, 4, 3)
(36, 4, 3)
(31, 

(45, 4, 3)
(32, 4, 3)
(40, 4, 3)
(34, 4, 3)
(37, 4, 3)
(35, 4, 3)
(55, 4, 3)
(37, 4, 3)
(38, 4, 3)
(34, 4, 3)
(36, 4, 3)
(25, 4, 3)
(25, 4, 3)
(11, 4, 3)
(3, 4, 3)
(28, 4, 3)
(10, 4, 3)
(12, 4, 3)
(20, 4, 3)
(31, 4, 3)
(34, 4, 3)
(28, 4, 3)
(89, 4, 3)
(90, 4, 3)
(21, 4, 3)
(22, 4, 3)
(12, 4, 3)
(13, 4, 3)
(74, 4, 3)
(28, 4, 3)
(26, 4, 3)
(27, 4, 3)
(34, 4, 3)
(39, 4, 3)
(24, 4, 3)
(22, 4, 3)
(25, 4, 3)
(14, 4, 3)
(25, 4, 3)
(32, 4, 3)
(26, 4, 3)
(33, 4, 3)
(26, 4, 3)
(33, 4, 3)
(26, 4, 3)
(24, 4, 3)
(21, 4, 3)
(5, 4, 3)
(32, 4, 3)
(27, 4, 3)
(45, 4, 3)
(33, 4, 3)
(27, 4, 3)
(62, 4, 3)
(13, 4, 3)
(32, 4, 3)
(6, 4, 3)
(6, 4, 3)
(43, 4, 3)
(35, 4, 3)
(15, 4, 3)
(22, 4, 3)
(20, 4, 3)
(13, 4, 3)
(13, 4, 3)
(4, 4, 3)
(6, 4, 3)
(8, 4, 3)
(19, 4, 3)
(29, 4, 3)
(22, 4, 3)
(24, 4, 3)
(80, 4, 3)
(21, 4, 3)
(48, 4, 3)
(14, 4, 3)
(17, 4, 3)
(22, 4, 3)
(18, 4, 3)
(15, 4, 3)
(32, 4, 3)
(32, 4, 3)
(9, 4, 3)
(13, 4, 3)
(13, 4, 3)
(14, 4, 3)
(35, 4, 3)
(36, 4, 3)
(17, 4, 3)
(17, 4, 3)
(17, 4, 3)
(22, 4,

(41, 4, 3)
(25, 4, 3)
(38, 4, 3)
(25, 4, 3)
(28, 4, 3)
(18, 4, 3)
(17, 4, 3)
(26, 4, 3)
(46, 4, 3)
(39, 4, 3)
(42, 4, 3)
(29, 4, 3)
(29, 4, 3)
(25, 4, 3)
(22, 4, 3)
(23, 4, 3)
(23, 4, 3)
(23, 4, 3)
(47, 4, 3)
(47, 4, 3)
(43, 4, 3)
(43, 4, 3)
(43, 4, 3)
(43, 4, 3)
(53, 4, 3)
(55, 4, 3)
(46, 4, 3)
(48, 4, 3)
(55, 4, 3)
(12, 4, 3)
(60, 4, 3)
(58, 4, 3)
(79, 4, 3)
(76, 4, 3)
(78, 4, 3)
(79, 4, 3)
(77, 4, 3)
(77, 4, 3)
(75, 4, 3)
(76, 4, 3)
(78, 4, 3)
(77, 4, 3)
(78, 4, 3)
(76, 4, 3)
(44, 4, 3)
(63, 4, 3)
(40, 4, 3)
(13, 4, 3)
(39, 4, 3)
(38, 4, 3)
(39, 4, 3)
(42, 4, 3)
(16, 4, 3)
(22, 4, 3)
(17, 4, 3)
(17, 4, 3)
(33, 4, 3)
(34, 4, 3)
(37, 4, 3)
(30, 4, 3)
(15, 4, 3)
(30, 4, 3)
(30, 4, 3)
(31, 4, 3)
(30, 4, 3)
(33, 4, 3)
(41, 4, 3)
(34, 4, 3)
(54, 4, 3)
(42, 4, 3)
(42, 4, 3)
(39, 4, 3)
(23, 4, 3)
(23, 4, 3)
(23, 4, 3)
(28, 4, 3)
(28, 4, 3)
(27, 4, 3)
(26, 4, 3)
(29, 4, 3)
(25, 4, 3)
(25, 4, 3)
(22, 4, 3)
(19, 4, 3)
(19, 4, 3)
(20, 4, 3)
(19, 4, 3)
(33, 4, 3)
(33, 4, 3)
(55, 4, 3)
(53, 4, 3)

# 删除有问题的数据

In [42]:
#删除有问题的数据
error_list = []
with open("eooro.txt", "r+", encoding="utf8") as f:
    lines= f.readline().strip()
    while lines:
        error_list.append(str(lines))
        lines= f.readline().strip()
print(error_list)
path = ['./pocket6A/','./pocket_pepbdb6A/','./pocket6A_gvp/','./peptide_pepbdb6A','./pocket_acid6A/','./peptide6A/']
for p in path:
    for file in error_list:
        new_path = str(p) + str(file)
        try:
            os.remove(new_path)
        except:
            continue
            

['1f8v_D.txt', '1h8d_L_H.txt', '1jq1_A.txt', '1jq2_A.txt', '1mue_A_B.txt', '1mue_C_B.txt', '1ofs_B_A.txt', '1ofs_D_A.txt', '1qcr_I.txt', '1qcr_K.txt', '1xyr_5.txt', '1xyr_6.txt', '1xyr_7.txt', '2bfx_C_B.txt', '2bfx_D_A.txt', '2bzw_B_A.txt', '3htc_L.txt', '3iyb_2.txt', '3iyb_7.txt', '3iyc_2.txt', '3iyc_7.txt', '3j3o_0.txt', '4cc7_D_A.txt', '4wol_A.txt', '5khr_S.txt', '5khu_U.txt', '5usv_A.txt', '6gvk_B_A.txt', '1rb6_A.txt', '1znj_A.txt', '2ljc_A.txt', '2o8m_C_A.txt', '3zqr_A.txt', '4unh_A.txt', '4wo1_A.txt', '5lvp_E_C.txt', '6tof_B_A.txt', '6tog_B_A.txt', '6v2d_B_A.txt', '173d_D.txt', '185d_A.txt', '193d_C.txt', '1a4t_B.txt', '1a6b_B.txt', '1biv_B.txt', '1bj6_A.txt', '1dsc_C.txt', '1dsd_C.txt', '1efg_C.txt', '1etf_B.txt', '1etg_B.txt', '1exy_B.txt', '1fe1_E.txt', '1fe1_F.txt', '1fja_C.txt', '1g70_B.txt', '1hji_B.txt', '1hvn_E.txt', '1hvo_E.txt', '1i3w_E.txt', '1i9f_B.txt', '1ilx_E.txt', '1ilx_F.txt', '1j9n_C.txt', '1jgo_X.txt', '1jgp_X.txt', '1jgq_X.txt', '1l1v_B.txt', '1ml5_X.txt', '1m